In [18]:
import pandas as pd
import requests
import os
from bs4 import BeautifulSoup
pd.set_option('display.max_rows', None)

In [19]:
headers = {'user-agent': 'Mozilla/5.0'}
resposta = requests.get('https://www.camarabotucatu.sp.gov.br/Vereador',
                        headers= headers
                        )

## Pegando o Nome e o Partido dos Vereadores

In [20]:
text_estruturado = BeautifulSoup(resposta.text)
#text_estruturado

In [21]:
lista_vereadores = text_estruturado.find_all('h2')

In [22]:
lista_vereadores

[<h2>
 <a href="http://www.camarabotucatu.sp.gov.br/Consulta/vereadores/Vereador/68" target="_blank">Abelardo Wanderlino da Costa Neto - Abelardo</a>
 </h2>,
 <h2>
 <a href="http://www.camarabotucatu.sp.gov.br/Consulta/vereadores/Vereador/81" target="_blank">Alessandra Lucchesi de Oliveira - Alessandra Lucchesi</a>
 </h2>,
 <h2>
 <a href="http://www.camarabotucatu.sp.gov.br/Consulta/vereadores/Vereador/69" target="_blank">André Rogério Barbosa - Curumim (Licenciado)</a>
 </h2>,
 <h2>
 <a href="http://www.camarabotucatu.sp.gov.br/Consulta/vereadores/Vereador/47" target="_blank">Antonio Carlos Vaz de Almeida - Cula</a>
 </h2>,
 <h2>
 <a href="http://www.camarabotucatu.sp.gov.br/Consulta/vereadores/Vereador/7076" target="_blank">Cláudia Maria Gabriel - Cláudia Gabriel (Licenciada)</a>
 </h2>,
 <h2>
 <a href="http://www.camarabotucatu.sp.gov.br/Consulta/vereadores/Vereador/7075" target="_blank">Elias Marcelo Sleiman - Marcelo Sleiman</a>
 </h2>,
 <h2>
 <a href="http://www.camarabotucatu.sp

In [23]:
# Captando o nome e o partido dos vereadores
tags_li = text_estruturado.find_all('span')
lista_partidos = []
for i in range(len(tags_li)):
    if tags_li[i].find_all(['li']):
        if tags_li[i].find_all('a'):
            lista_partidos.append( tags_li[i].text.replace('\n', '') )


nomes_vereadores = []
for i in range(len(lista_vereadores)):
    #print(i)
    nomes_vereadores.append(  lista_vereadores[i].contents[1].text.split('-')[1].strip()  )


## Captando as imagens dos Vereadores

In [24]:
import re

teste = []
teste2 = []

x = text_estruturado.find_all('div', {'class': 'col-md-3'})
for i in range(len(x)):
    if x[i].find('div', {'class':'img'}):
        teste.append(x[i].find('div', {'class':'img'}))
        #print(teste)
for j in range(len(teste)):
    if teste[j].find('img'):
        teste2.append(teste[j].find_all('img', {'src': re.compile('\.jpg$', re.IGNORECASE)}))


In [25]:
save_folder = "../img"
for img_tag in teste2:
    src = img_tag[0]['src']
    alt = img_tag[0]['alt'].split('-')[1].strip()
    img_url = f"https://www.camarabotucatu.sp.gov.br/{src}" 
    filename = os.path.join(save_folder, f"{alt}.jpg")

    response = requests.get(img_url)
    with open(filename, 'wb') as f:
        f.write(response.content)

In [26]:
df_vereadores = pd.DataFrame()

In [27]:
lista_partidos

['Republicanos',
 'PSDB-Partido da Social Democracia Brasileira',
 'PSDB-Partido da Social Democracia Brasileira',
 'PSDB-Partido da Social Democracia Brasileira',
 'União Brasil',
 'União Brasil',
 'Republicanos',
 'União Brasil',
 'PSDB-Partido da Social Democracia Brasileira',
 'PSDB-Partido da Social Democracia Brasileira',
 'União Brasil',
 'PDT-Partido Democrático Trabalhista',
 'Republicanos']

In [28]:
df_vereadores['nome_vereador'] = nomes_vereadores
df_vereadores['partido_vereador'] = lista_partidos

In [29]:
df_vereadores

,nome_vereador,partido_vereador
0,Abelardo,Republicanos
1,Alessandra Lucchesi,PSDB-Partido da Social Democracia Brasileira
2,Curumim (Licenciado),PSDB-Partido da Social Democracia Brasileira
3,Cula,PSDB-Partido da Social Democracia Brasileira
4,Cláudia Gabriel (Licenciada),União Brasil
5,Marcelo Sleiman,União Brasil
6,Erika da Liga do Bem,Republicanos
7,Pedroso (Suplente),União Brasil
8,Sargento Laudo,PSDB-Partido da Social Democracia Brasileira
9,Lelo Pagani (Suplente),PSDB-Partido da Social Democracia Brasileira


## Para Cada Vereador trazer tabela do que ja fez em cada Ano

In [30]:
links_vereadores = []
for i in range(len(lista_vereadores)):
    links_vereadores.append(lista_vereadores[i].a.get('href'))

df_vereadores['link'] = links_vereadores

In [31]:
df_final = pd.DataFrame(columns=['Tipo', 'Quantidade','Ano'])

for i in range(len(df_vereadores)):

    print(i)

    #pega o nome do vereador
    nome_atual_vereador = df_vereadores.iloc[i][:i+1]['nome_vereador']


    #pega o link do vereador
    #df_vereadores.iloc[i][:i+1]['link']


    headers = {'user-agent': 'Mozilla/5.0'}
    resposta_vereador = requests.get(df_vereadores[:i+1]['link'].iloc[-1],
                        headers= headers
                        )
    
    vereador_estruturado = BeautifulSoup(resposta_vereador.text)
    tabela_documentos = vereador_estruturado.find_all('table')

    # Pegando a linha da tabela do site que contém os anos de atuação dos vereadores
    lista_anos = []

    for i in range(len(tabela_documentos)):
        if tabela_documentos[i].find_all('caption')[0].text == 'Documentos':
            tabela_documentos = tabela_documentos[i]


    for i in range(len( tabela_documentos.find_all('th', {'class': 'text-right'}) ) ):
        if tabela_documentos.find_all('th', {'class': 'text-right'})[i].text:
            lista_anos.append(tabela_documentos.find_all('th', {'class': 'text-right'})[i].text)
    lista_anos = sorted([*set(lista_anos)])
    if 'Total' in lista_anos:
        lista_anos.remove('Total')
    #lista_anos


    # Pegando todas as proposituras do vereador em cada tipo e alocando em um dataframe
    tbody = tabela_documentos.find_all('tbody')[0].find_all('tr')#[0]#.find_all('td')
    list_dict = []
    df_teste = pd.DataFrame(columns=['Tipo', 'Quantidade'])

    for i in range(len(tbody)):
        for j in range(len( tbody[i].find_all('td') )):
            list_dict.append(tbody[i].find_all('td')[j].text)
        #print(list_dict)
        
        #list_aux = list_dict
        #list_aux.remove(list_aux[0])
        df_novo = pd.DataFrame({'Tipo': [list_dict[0]]*(len(list_dict)-2), 'Quantidade': list_dict[1:-1], 'Ano': lista_anos,'Vereador': nome_atual_vereador})
        df_teste = pd.concat([df_teste, df_novo], ignore_index=True)

        list_dict = []
    df_final = pd.concat([df_final,df_teste])
    df_final = df_final.loc[df_final['Tipo'] != 'Total']
    
    

0
1


In [ ]:
df_final['Quantidade'] = df_final['Quantidade'].apply(lambda x: 0 if x == '-' else x)
df_final.head()

,Tipo,Quantidade,Ano,Vereador
0,Correspondências Recebidas,0,2009,Abelardo
1,Correspondências Recebidas,1,2010,Abelardo
2,Correspondências Recebidas,0,2011,Abelardo
3,Correspondências Recebidas,0,2012,Abelardo
4,Correspondências Recebidas,0,2017,Abelardo
